In [1]:
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/NLP/project

Mounted at /gdrive
/gdrive/My Drive/NLP/project


In [140]:
%pip install transformers datasets ray[tune] rouge bert-score

In [3]:
!nvidia-smi

Mon Nov 29 21:54:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [141]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import functional as F
from collections import Counter, namedtuple

import pandas as pd
import seaborn as sns
import numpy as np
import random
import math

import matplotlib.pyplot as plt

from transformers import GPT2LMHeadModel,  GPT2TokenizerFast, GPT2Config, DataCollatorForLanguageModeling, LineByLineTextDataset
from transformers import AdamW, top_k_top_p_filtering
from datasets import load_metric

from tqdm import tqdm
import nltk
from nltk.util import ngrams
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from bert_score import score as BERTscore
from rouge import Rouge

import gpt2
from dataset import *

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training

## Elon Musk

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/elonmusk \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/elonmusk/train.txt \
  --eval_data_file=data/elonmusk/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

Streaming output truncated to the last 5000 lines.
Memory usage on this node: 7.0/25.5 GiB
Using FIFO scheduling algorithm.
Resources requested: 1.0/4 CPUs, 1.0/1 GPUs, 0.0/15.02 GiB heap, 0.0/7.51 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2021-11-19_21-22-15
Number of trials: 10/10 (7 PENDING, 1 RUNNING, 2 TERMINATED)
+------------------------+------------+-----------------+-------------------------------+-----------------+-------------------------------+-------------+
| Trial name             | status     | loc             |   gradient_accumulation_steps |   learning_rate |   per_device_train_batch_size |   objective |
|------------------------+------------+-----------------+-------------------------------+-----------------+-------------------------------+-------------|
| _objective_c48e9_00002 | RUNNING    | 172.28.0.2:1052 |                             1 |     7.87566e-05 |                             2 |             |
| _objective_c48e

In [8]:
!python run_language_modeling.py --output_dir=models/elonmusk --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/elonmusk/train.txt --eval_data_file=data/elonmusk/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

11/29/2021 21:57:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
11/29/2021 21:57:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Andrej Karpathy

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/karpathy \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/karpathy/train.txt \
  --eval_data_file=data/karpathy/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

Streaming output truncated to the last 5000 lines.


(ImplicitFunc pid=1551) {'train_runtime': 159.618, 'train_samples_per_second': 50.22, 'train_steps_per_second': 12.555, 'train_loss': 3.5625168901241704, 'epoch': 3.0}
Result for _objective_3640f_00003:
  date: 2021-11-19_22-01-21
  done: true
  epoch: 3.0
  eval_loss: 3.7516210079193115
  eval_runtime: 2.4525
  eval_samples_per_second: 311.112
  eval_steps_per_second: 39.144
  experiment_id: 2418950cb1e841a08e0c33a14d9b6724
  hostname: b5c728523e37
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 3.7516210079193115
  pid: 1551
  time_since_restore: 168.7035608291626
  time_this_iter_s: 168.7035608291626
  time_total_s: 168.7035608291626
  timestamp: 1637359281
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3640f_00003
  
(ImplicitFunc pid=1551) {'eval_loss': 3.7516210079193115, 'eval_runtime': 2.4525, 'eval_samples_per_second': 311.112, 'eval_steps_per_second': 39.144, 'epoch': 3.0}
(raylet) /usr/lo

In [ ]:
!python run_language_modeling.py --output_dir=models/karpathy --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/karpathy/train.txt --eval_data_file=data/karpathy/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=5

10/26/2021 02:16:07 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:16:07 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## Barack Obama

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/BarackObama \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/BarackObama/train.txt \
  --eval_data_file=data/BarackObama/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
T!python run_language_modeling.py --output_dir=models/BarackObama --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/BarackObama/train.txt --eval_data_file=data/BarackObama/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=3

10/26/2021 02:26:06 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 02:26:06 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## WSJ

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/wsj \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/wsj/train.txt \
  --eval_data_file=data/wsj/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
!python run_language_modeling.py --output_dir=models/wsj --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/wsj/train.txt --eval_data_file=data/wsj/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:16:38 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:16:38 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

## CNN

In [ ]:
!python run_hp_search.py \
  --output_dir=hp_search_results/cnn \
  --overwrite_output_dir \
  --model_type=gpt2 \
  --model_name_or_path=gpt2 \
  --do_train \
  --train_data_file=data/cnn/train.txt \
  --eval_data_file=data/cnn/valid.txt \
  --do_eval \
  --disable_tqdm=True \
  --line_by_line

In [ ]:
!python run_language_modeling.py --output_dir=models/cnn --overwrite_output_dir --model_type=gpt2 --model_name_or_path=gpt2 --do_train --train_data_file=data/cnn/train.txt --eval_data_file=data/cnn/valid.txt --do_eval --per_device_train_batch_size=2 --per_device_eval_batch_size=2 --line_by_line --learning_rate=5e-5 --num_train_epochs=2 --save_strategy=epoch

10/26/2021 03:40:09 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
10/26/2021 03:40:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.0,
learning_rate=5e-05,
length_column_name=length,
load_best_model_

# Generation and Metrics

In [172]:
def read_line_by_line_dataset(file_path, tokenize=False):
    with open(file_path, encoding="utf-8") as f:
        lines = [line.split()[1:-1] for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
    
    if tokenize:
        return lines
    return [" ".join(line) for line in lines]


def compute_bleu_score(username, num_ref=5):
    ref_dataset = read_line_by_line_dataset(f'data/{username}/test.txt', tokenize=True)
    references = [ref_dataset[i] for i in range(num_ref)]
    scores = []
    for reference in references:
        ref_len = len(reference)
        candidate = gpt2.generate(username, prompt="<BOS> " + reference[0], min_length=ref_len).split()
        score = nltk.bleu_score.sentence_bleu(references, candidate)
        scores.append(score)
    return np.mean(np.array(scores))

def compute_rouge_score(username, num_ref=5):
    rouge = Rouge()
    ref_dataset = read_line_by_line_dataset(f'data/{username}/test.txt')
    references = [ref_dataset[i] for i in range(num_ref)]
    candidates = []
    for reference in references:
        reference = reference.split()
        ref_len = len(reference)
        candidate = gpt2.generate(username, prompt="<BOS> " + reference[0], min_length=ref_len)
        candidates.append(candidate)
    return rouge.get_scores(candidates, references, avg=True)

def compute_bert_score(username, num_ref=5):
    ref_dataset = read_line_by_line_dataset(f'data/{username}/test.txt')
    references = [ref_dataset[i] for i in range(num_ref)]
    candidates = []
    for reference in references:
        reference = reference.split()
        ref_len = len(reference)
        candidate = gpt2.generate(username, prompt="<BOS> " + reference[0], min_length=ref_len)
        candidates.append(candidate)
    P, R, F1 = BERTscore(candidates, references, lang="en")
    print(f"BERTscore: precision={P.mean().item():.6f} recall={R.mean().item():.6f} F1={F1.mean().item():.6f}")
    return P, R, F1

## Elon Musk

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/elonmusk" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:28 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/elonmusk', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Am excited about the Model 3 unveil and road trip this weekend. There are some huge changes coming to the design as well, so this one is easy & fast. 
=== GENERATED SEQUENCE 2 ===
To be clear, I am against artificial intelligence. AI should not be in charge. I want it to be. 
=== GENERATED SEQUENCE 3 ===
The last remaining soul mate of the late King Arthur was actually Dragan Motavalli. He was one of the greats. 
=== GENERATED SEQUENCE 4 ===
My kids love  @SpaceX  and  @NASA. 
=== GENERATED SEQUENCE 5 ===
It's not just me: I have several cats. I also own a chicken

In [173]:
elon_bleu_score = compute_bleu_score('elonmusk')
elon_bleu_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.4195736504608999

In [174]:
elon_bert_score = compute_bert_score('elonmusk')
elon_bert_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

BERTscore: precision=0.819619 recall=0.835668 F1=0.827541


(tensor([0.8249, 0.8177, 0.8146, 0.8313, 0.8096]),
 tensor([0.8583, 0.8246, 0.8261, 0.8487, 0.8205]),
 tensor([0.8412, 0.8212, 0.8203, 0.8399, 0.8150]))

In [175]:
elon_rouge_score = compute_rouge_score('elonmusk')
elon_rouge_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge-1': {'f': 0.10091454761326372,
  'p': 0.07652618135376757,
  'r': 0.1587675070028011},
 'rouge-2': {'f': 0.028571427653061254, 'p': 0.02222222222222222, 'r': 0.04},
 'rouge-l': {'f': 0.10091454761326372,
  'p': 0.07652618135376757,
  'r': 0.1587675070028011}}

## Andrej Karpathy

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/karpathy" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:40 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/karpathy', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
"How to Start a Startup" from  @OpenAI  great resource, tips & tricks! 
=== GENERATED SEQUENCE 2 ===
A new Deep Learning tutorial for beginners:  the DeepMind GTC231n video   (w)’s A New Challenge for Computer Visionists 
=== GENERATED SEQUENCE 3 ===
The Deep Web is a Hotbed of Violence and Corruption  soooo excited! 
=== GENERATED SEQUENCE 4 ===
#cvpr2016  a week of hacking on the ConvNetCVPR code. Looks like I'm on a roll with the demos 
=== GENERATED SEQUENCE 5 ===
It's quite amusing to watch what is happening behind closed doors with no public hearings or publ

In [176]:
karpathy_bleu_score = compute_bleu_score('karpathy')
karpathy_bleu_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.48204379554011745

In [177]:
karpathy_bert_score = compute_bert_score('karpathy')
karpathy_bert_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

BERTscore: precision=0.831352 recall=0.823897 F1=0.827574


(tensor([0.8340, 0.8338, 0.8473, 0.8319, 0.8098]),
 tensor([0.8096, 0.8387, 0.8383, 0.8204, 0.8124]),
 tensor([0.8216, 0.8363, 0.8428, 0.8261, 0.8111]))

In [178]:
karpathy_rouge_score = compute_rouge_score('karpathy')
karpathy_rouge_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge-1': {'f': 0.10535459496724561,
  'p': 0.1141025641025641,
  'r': 0.10809523809523809},
 'rouge-2': {'f': 0.007999999001600125,
  'p': 0.007692307692307693,
  'r': 0.008333333333333333},
 'rouge-l': {'f': 0.10535459496724561,
  'p': 0.1141025641025641,
  'r': 0.10809523809523809}}

## Barack Obama

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/BarackObama" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 03:01:52 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/BarackObama', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
“They’re going to try to scare us off education entirely, because no one wants to go to college.” 
=== GENERATED SEQUENCE 2 ===
"That said, I know that people can still make their voices heard in Washington. I know it's not easy to do that." —President Obama 
=== GENERATED SEQUENCE 3 ===
The Senate must make a timely vote on an American Jobs Act now. The Jobs Act needs to be included in any deal. 
=== GENERATED SEQUENCE 4 ===
There's still more work to do, but this is progress. 
=== GENERATED SEQUENCE 5 ===
It's up to us to work with leaders in both parties to 

In [179]:
obama_bleu_score = compute_bleu_score('BarackObama')
obama_bleu_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.44386069378185206

In [180]:
obama_bert_score = compute_bert_score('BarackObama')
obama_bert_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

BERTscore: precision=0.842415 recall=0.850196 F1=0.845924


(tensor([0.8581, 0.8551, 0.8022, 0.8401, 0.8566]),
 tensor([0.8212, 0.8531, 0.8716, 0.8537, 0.8514]),
 tensor([0.8392, 0.8541, 0.8354, 0.8468, 0.8540]))

In [181]:
obama_rouge_score = compute_rouge_score('BarackObama')
obama_rouge_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge-1': {'f': 0.10934239972403188,
  'p': 0.13180205415499532,
  'r': 0.1567020202020202},
 'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
 'rouge-l': {'f': 0.10934239972403188,
  'p': 0.13180205415499532,
  'r': 0.1567020202020202}}

## CNN

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/cnn" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:04:59 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/cnn', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
Obama: The "threat" to national security from North Korea is not unique, "from my conversations with all my colleagues, from this administration, and from the President himself" 
=== GENERATED SEQUENCE 2 ===
A new study finds that babies born with a rare form of brain cancer can develop better health than their non-nuclear counterparts 
=== GENERATED SEQUENCE 3 ===
The Senate has passed a resolution to censure President Trump for calling former President Joe Biden a 'narcissistic prick' 
=== GENERATED SEQUENCE 4 ===
 @HillaryClinton  and Donald Trump campaign to attend

In [182]:
cnn_bleu_score = compute_bleu_score('cnn')
cnn_bleu_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.5407421909800207

In [183]:
cnn_bert_score = compute_bert_score('cnn')
cnn_bert_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

BERTscore: precision=0.875326 recall=0.885970 F1=0.880475


(tensor([0.8876, 0.8994, 0.8588, 0.8912, 0.8396]),
 tensor([0.8911, 0.8893, 0.9125, 0.8993, 0.8376]),
 tensor([0.8894, 0.8944, 0.8849, 0.8952, 0.8386]))

In [184]:
cnn_rouge_score = compute_rouge_score('cnn')
cnn_rouge_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge-1': {'f': 0.16013513448003988,
  'p': 0.16367880485527544,
  'r': 0.18484848484848487},
 'rouge-2': {'f': 0.05541125347620929,
  'p': 0.05833333333333333,
  'r': 0.055384615384615386},
 'rouge-l': {'f': 0.16013513448003988,
  'p': 0.16367880485527544,
  'r': 0.18484848484848487}}

## WSJ

In [ ]:
!python run_generation.py \
  --model_type gpt2 \
  --model_name_or_path "models/wsj" \
  --length 50 \
  --prompt "<BOS>" \
  --stop_token "<EOS>" \
  --k 50 \
  --num_return_sequences 5

10/26/2021 04:05:10 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=50, length=50, model_name_or_path='models/wsj', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=5, p=0.9, padding_text='', prompt='<BOS>', repetition_penalty=1.0, seed=42, stop_token='<EOS>', temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
‘They’re going to attack again, and it’s going to be a real bloodbath,’ says one Republican in Tennessee. ‘It’s going to be devastating.’ <
=== GENERATED SEQUENCE 2 ===
A new study finds that the number of overweight people is rising.  via  @WSJNY 
=== GENERATED SEQUENCE 3 ===
The Senate tax bill was expected to pass Thursday, but Democrats are delaying key provisions 
=== GENERATED SEQUENCE 4 ===
Opinion: Why the Supreme Court hasn’t heard the case on gay marriage. writes  @wjmcgurn 
=== GENERATED SEQUENCE 5 ===
How to be more ambitious: 


In [185]:
wsj_bleu_score = compute_bleu_score('cnn')
wsj_bleu_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0.6332040444333786

In [186]:
wsj_bert_score = compute_bert_score('wsj')
wsj_bert_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceCl

BERTscore: precision=0.911430 recall=0.895396 F1=0.903222


(tensor([0.8627, 0.8788, 1.0000, 0.9280, 0.8876]),
 tensor([0.8621, 0.8276, 1.0000, 0.9021, 0.8852]),
 tensor([0.8624, 0.8524, 1.0000, 0.9149, 0.8864]))

In [187]:
wsj_rouge_score = compute_rouge_score('wsj')
wsj_rouge_score

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'rouge-1': {'f': 0.3231111063936113,
  'p': 0.3296969696969697,
  'r': 0.33157088122605366},
 'rouge-2': {'f': 0.24210526116066483, 'p': 0.24444444444444446, 'r': 0.24},
 'rouge-l': {'f': 0.3231111063936113,
  'p': 0.3296969696969697,
  'r': 0.33157088122605366}}